In [1]:
import numpy as np
from networkx import nx
from collections import defaultdict

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import nxtriads as nx2 ## import modified functions

In [2]:
# edgesAfter = pd.read_csv("../data/test.csv")

edgesAfter = pd.read_csv("../data/votes_to_comments_after.csv")
edgesAfter = edgesAfter.loc[9000:9900,:]

In [3]:
G = nx.from_pandas_edgelist(edgesAfter, 
                            source='source', 
                            target='target', 
                            edge_attr = 'weight',
                            create_using=nx.DiGraph())

In [4]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 736\nNumber of edges: 901\nAverage in degree:   1.2242\nAverage out degree:   1.2242'

In [5]:
x = nx.triadic_census(G)
x

{'003': 65543693,
 '012': 606174,
 '102': 0,
 '021D': 26887,
 '021U': 256,
 '021C': 416,
 '111D': 0,
 '111U': 0,
 '030T': 14,
 '030C': 0,
 '201': 0,
 '120D': 0,
 '120U': 0,
 '120C': 0,
 '210': 0,
 '300': 0}

In [6]:
def triads_by_specific_types(G, my_types):
    """Returns a list of triads of pre-defined types in a directed graph.
    Parameters
    ----------
    G : digraph
       A NetworkX DiGraph
    Returns
    -------
    tri_by_type : dict
       Dictionary with triad types as keys and lists of triads as values.
    """
    all_tri = nx2.all_triads_mod(G) # modified function to skip if len < 3
    tri_by_type = defaultdict(list)
    for triad in all_tri:
        tritype = nx2.triad_type_mod(triad) ## removed duplicate call, modified fx to skip calls for < 3
        if (tritype in my_types):
            tri_by_type[tritype].append(triad)
            print(tritype)
    return tri_by_type

In [7]:
test = triads_by_specific_types(G, my_types = {'030T', '120D', '120U', '300'})
#(test['120U'])[:5]
## works but takes forever on edges=1000 already

030T
030T
030T
030T
030T
030T
030T
030T
030T
030T


KeyboardInterrupt: 

In [34]:
(test['030T'])

[]

In [32]:
len(test['021D'])

1225

In [171]:
def balance_ratio(graph):
    my_types = {'030T', '120D', '120U', '300'}
    triads = triads_by_specific_types(graph, my_types)
    
#    for type in my_types:
#        for i in range(len(triads[type])):
            
    
    
    return triads
    
    

In [172]:
balance_ratio(G)

defaultdict(list,
            {'030T': [<networkx.classes.digraph.DiGraph at 0x2d683483308>],
             '120U': [<networkx.classes.digraph.DiGraph at 0x2d6834833c8>],
             '120D': [<networkx.classes.digraph.DiGraph at 0x2d6834832c8>],
             '300': [<networkx.classes.digraph.DiGraph at 0x2d6834834c8>]})

In [37]:
y = nx2.triads_by_type(G)
blubb = (y['120D'])[0].edges(data = True)
blubb

IndexError: list index out of range

In [183]:
((y['120D'])[0].get_edge_data(2, 5))['weight']

1

In [184]:
for i in blubb:
    print((i[2])['weight'])

1
1
1
1
